# Syntactic analysis

## 1. Explain the meaning of the three UD tags of syntactic relations using a fragment of a marked-up corpus as an example

In [1]:
! wget -q https://www.dropbox.com/s/am6nasx6bx82nhp/RuEval2017-Lenta-news-dev.conllu

In [3]:
! head -497 RuEval2017-Lenta-news-dev.conllu | tail -10

21	в	в	ADP	_	_	23	case	_	_
22	1969	1969	ADJ	_	NumForm=Digit	23	amod	_	_
23	году	год	NOUN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	20	obl	_	_
24	за	за	ADP	_	_	25	case	_	_
25	задержание	задержание	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	20	obl	_	_
26	особо	особо	ADV	_	Degree=Pos	27	advmod	_	_
27	опасного	опасный	ADJ	_	Case=Gen|Degree=Pos|Gender=Masc|Number=Sing	28	amod	_	_
28	преступника	преступник	NOUN	_	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	25	nmod	_	_
29	.	.	PUNCT	_	_	9	punct	_	_



In [10]:
corpus_file = 'RuEval2017-Lenta-news-dev.conllu'

In [11]:
! pip install -q natasha
! pip install -q conllu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [53]:
from natasha.syntax import NewsSyntaxParser
from natasha import (
    Doc, 
    Segmenter,
    NewsEmbedding,
    # MorphVocab,
    # NewsMorphTagger
)
from conllu import parse
import string

# Create instances of needed Natasha components
segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)
# morph_vocab = MorphVocab()
# morph_tagger = NewsMorphTagger(emb)

# Download CoNLL-U
with open(corpus_file, 'r', encoding='utf-8') as file:
    conll_text = file.read()

# Parse CoNLL-U text and create Doc object from tokens
parsed_data = parse(conll_text)
# tokens = [token['form'] for sentence in parsed_data for token in sentence if token['form'] not in string.punctuation]
tokens = [token['form'] for sentence in parsed_data for token in sentence]
print(tokens[5:8])
doc = Doc(' '.join(tokens))
# Tokenization
doc.segment(segmenter)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()
# print(doc.tokens[5:8])
# # Setting morphological tags for tokens
# doc.tag_morph(morph_tagger)
# # Getting tokens and morphological tags
# for token in doc.tokens:
#     print("Токен: ", token.text)
#     print("Тег: ", token.pos)
#     print("Тег морфологии: ", token.feats)
#     print("--------------")

['подробности', 'программы', ',']
    ┌──► Официальные    amod
    │ ┌► американские   amod
    └─└─ власти         nsubj
┌───└─┌─ отказываются   
│ ┌─┌─└► комментировать xcomp
│ │ └►┌─ подробности    obj
│ │   └► программы      nmod
│ │   ┌► ,              punct
│ └──►└─ ссылаясь       advcl
│ │ ┌──► на             case
│ │ │ ┌► ее             det
│ └►└─└─ секретность    obl
└──────► .              punct


## 2. Write a function for splitting a two-part compound sentence into simple ones

### razdel

In [15]:
! pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
text = '''
- "Так в чем же дело?" - "Не ра-ду-ют".
И т. д. и т. п. В общем, вся газета
'''

from razdel import sentenize
print(list(sentenize(text)))
# print(list(sentenize(doc.sents[0].text)))

[Substring(1, 23, '- "Так в чем же дело?"'), Substring(24, 40, '- "Не ра-ду-ют".'), Substring(41, 56, 'И т. д. и т. п.'), Substring(57, 76, 'В общем, вся газета')]


In [58]:
sent1 = doc.sents[0]
sent2 = doc.sents[1]
print(sent1.text)
print(sent2.text)

Официальные американские власти отказываются комментировать подробности программы , ссылаясь на ее секретность .
Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас находится в руках американской разведки .


### spacy

In [32]:
! pip install -q spacy
! python -m spacy download ru_core_news_sm --quiet

2023-04-19 00:02:29.129034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [60]:
import spacy

nlp = spacy.load("ru_core_news_sm")

doc_spacy = nlp(sent1.text)

simple_sentences = [sent.text for sent in doc_spacy.sents]

for i, simple_sentence in enumerate(simple_sentences):
    print(f"Простое предложение {i + 1}: {simple_sentence}")

Простое предложение 1: Официальные американские власти отказываются комментировать подробности программы , ссылаясь на ее секретность .


### custom

In [106]:
import string

def split_complex_sentences(doc):
    # Split complex sentences into simple clauses
    j = 0
    split_sentences = []
    for sent in doc.sents:
        clauses = []
        conj_indices = [i for i, token in enumerate(sent.tokens) if token.rel == 'conj']
        if conj_indices:
            print(sent.text)
            clause_start = 0
            for conj_index in conj_indices:
                clause_end = conj_index
                clause = ' '.join(token.text for token in sent.tokens[clause_start:clause_end])
                if clause:
                    clauses.append(clause.strip())
                clause_start = conj_index
            last_clause = ' '.join(token.text for token in sent.tokens[conj_indices[-1]:])
            if last_clause:
                clauses.append(last_clause.strip())
            split_sentences.extend(clauses)
            for i, clause in enumerate(clauses):
              print(r'Simple sentence {}: {}'.format(i + 1, clause))
            j += 1
            if j > 9:
              break
            print('')

    return split_sentences

In [107]:
split_sentences = split_complex_sentences(doc)

Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас находится в руках американской разведки .
Simple sentence 1: Единственный сын одного из высокопоставленных северокорейских генералов бежал из страны вместе с семьей и сейчас
Simple sentence 2: находится в руках американской разведки .

Это решение уже трижды рассматривалось в судах — первая инстанция удовлетворила жалобу " ЮКОСа " , однако вторая и третья признали арест акций законным .
Simple sentence 1: Это решение уже трижды рассматривалось в судах — первая инстанция удовлетворила жалобу " ЮКОСа " , однако вторая и
Simple sentence 2: третья признали арест акций законным .

Ранее один из преступников , представившийся " Хасаном " , заявил , что он " русский и требует самолет в Россию " .
Simple sentence 1: Ранее один из преступников , представившийся " Хасаном " , заявил , что он " русский и
Simple sentence 2: требует самолет в Россию " .

Министр общественного порядка так